In [1]:
from cainiao_utils import *  # 引用寫好的 function
from collections import Counter
import datetime as dt
import gc
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import pathlib
import random
from scipy import stats
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')



In [2]:
# Variables
dataset_folderpath = 'Cainiao_dataset/'
preprocessed_folderpath = 'Cainiao_preprocessed/'
output_folderpath = 'Cainiao_output/'
order_filepath = 'msom_order_data_1.csv'
logistic_filepath = 'msom_logistic_detail_1.csv'
logistic_cols = ['order_id','order_date','logistic_order_id','action','facility_id','facility_type','city_id',
                 'logistic_company_id','timestamp']
order_cols = ['day','order_id','item_det_info','pay_timestamp','buyer_id','promise_speed','if_cainiao','merchant_id',
              'Logistics_review_score']

produce_date = str(dt.date.today()).replace('-', '')
sent_day = 10  # 0 ≤ sent_day ≤ 10
check_best_centers = False  # {True, False}
cluster_method = 'kmeans'  # {'hcut', 'kmeans'}
center = 2  # center ≥ 2
OM_version = 'OMspell'  # {'OM', 'OMloc', 'OMslen', 'OMspell', 'OMstran'}
sm_method = 'TRATE'
indel_method = 'auto'
file_name = f'{produce_date}-sentday_{sent_day}'



#### 創建資料夾及載入資料集

In [3]:
# 若本地端沒有該資料夾則則創建
for path in [preprocessed_folderpath, output_folderpath]:
    path = pathlib.Path(path)
    path.mkdir(parents=True, exist_ok=True)

# 載入訂單及物流進程資料集
order = pd.read_csv(dataset_folderpath + order_filepath, header=None)
order.columns = order_cols
order = order.drop(['day','item_det_info','buyer_id','merchant_id'], axis=1)
order = order[~order['Logistics_review_score'].isnull()]
logistic = pd.read_csv(dataset_folderpath + logistic_filepath, header=None)
logistic.columns = logistic_cols



### 資料前處理與特徵工程

#### 研究對象訂單抽樣
- 根據滿意度分數抽樣: 1分 / 5分 

In [4]:
# 一月份共 1400 多萬筆訂單，從評分 1-5 分各抽樣 3 萬筆訂單，減少運算時間
sample_oids = list()
for score in range(1, 6):
    review_oids = np.random.choice(a=order[order['Logistics_review_score']==score].order_id.unique(), size=30000, replace=False)
    sample_oids.append(review_oids)
sample_oids = np.concatenate(sample_oids)
order = order[order['order_id'].isin(sample_oids)]
print('Shape of order: ', order.shape)
order.head()

# 合併訂單和其進程紀錄
logistic = pd.merge(logistic, order, on='order_id', how='left')
logistic = logistic[~logistic['Logistics_review_score'].isnull()]
logistic = logistic.drop_duplicates()
print('Shape of logistic: ', logistic.shape)
logistic.head()

# 儲存備用
logistic.to_csv(dataset_folderpath + f'order_logistic_log-{produce_date}.csv', index=False)

# 節省空間將不用的訂單資料表刪除
del order
gc.collect()


Shape of order:  (150125, 5)
     order_id        pay_timestamp  promise_speed  if_cainiao  \
6       10383  2017-01-01 14:14:00            NaN           0   
44      83413  2017-01-01 16:55:00            2.0           1   
131    257624  2017-01-01 17:28:00            NaN           0   
220    435788  2017-01-01 19:26:00            NaN           0   
299    606332  2017-01-01 14:27:00            NaN           1   

     Logistics_review_score  
6                       3.0  
44                      5.0  
131                     3.0  
220                     5.0  
299                     3.0  
Shape of logistic:  (1671621, 13)
     order_id  order_date  logistic_order_id         action  facility_id  \
142  49778250    20170121           90186180      DEPARTURE          NaN   
143  49778250    20170121           90186180            GOT          NaN   
144  49778250    20170121           90186180  TRADE_SUCCESS          NaN   
145  49778250    20170121           90186180      SENT_SCAN   

0

- 根據運送天數篩選: 運送天數 >10 天


In [5]:
# 計算每張訂單的運送天數
df_sent_times = compute_sent_days(df=logistic)
print(df_sent_times.groupby(['sent_duration']).size().reset_index().rename(columns={0: 'count'}))
df_sent_times.to_csv(preprocessed_folderpath + 'cainiao-sent_times.csv', index=False)

# 篩選出運送天數 >10 天的訂單
sent_duration_10dayup = df_sent_times[df_sent_times['sent_duration']=='10+'].order_id.unique()
logistic = logistic[logistic['order_id'].isin(sent_duration_10dayup)]

# 依據滿意度評分各隨機抽樣 1000 筆訂單
sample_oids = list()
for score in [1, 5]:
    review_oids = np.random.choice(a=logistic[logistic['Logistics_review_score']==score].order_id.unique(),
                                   size=1000, replace=False)
    sample_oids.append(review_oids)
sample_oids = np.concatenate(sample_oids)
logistic = logistic[logistic['order_id'].isin(sample_oids)]

# 將抽樣後資料連同運送天數一起儲存備用
logistic = pd.merge(logistic, df_sent_times, on='order_id', how='left')
logistic.to_csv(preprocessed_folderpath + f'Cainiao-sampledData_reviewscore-{file_name}.csv', index=False)
logistic.head()


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 149972/149972 [06:22<00:00, 392.04it/s]


         order_id           start_time             end_time  sent_duration
797     132274307  2017-01-12 16:23:00  2017-01-09 18:11:00             -3
960      62902941  2017-01-19 17:45:00  2017-01-12 14:27:00             -8
1813    102874777  2017-01-18 18:56:00  2017-01-14 15:03:00             -5
2169     97437633  2017-01-18 21:06:00  2017-01-08 11:37:00            -11
2389     37874371  2017-01-08 10:38:00  2017-01-05 11:59:00             -3
...           ...                  ...                  ...            ...
144535   16685596  2017-01-17 16:19:00  2017-01-10 17:13:00             -7
144878   26072442  2017-01-13 11:04:00  2017-01-11 14:48:00             -2
147119   61339797  2017-01-09 10:21:00  2017-01-06 18:35:00             -3
147301   86458568  2017-01-26 10:02:00  2017-01-24 15:38:00             -2
148661   52022189  2017-01-16 22:23:00  2017-01-11 12:31:00             -6

[340 rows x 4 columns]
   sent_duration  count
0              0   3770
1              1  18535
2   

#### 將物流進程日誌轉換為狀態序列

In [23]:
# 計算物流狀態時長佔比，作為序列長度，將每筆訂單轉換為物流狀態序列
actions_collect, reviews_collect = generate_logistic_state_sequence(df=logistic, order_id_list=sample_oids)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 813.78it/s]


In [24]:
# 將物流狀態序列存為 .csv
action_cols = ['action_'+str(x) for x in range(0, pd.DataFrame(actions_collect).shape[1]-1)]
action_cols = ['order_id'] + action_cols
order_logistic_states = pd.DataFrame(actions_collect, columns=action_cols)
order_logistic_states['review_score'] = reviews_collect
order_logistic_states.to_csv(preprocessed_folderpath + f'order_logistic_states-{produce_date}-sentday_{sent_day}.csv', index=False)
print(order_logistic_states.groupby(['review_score']).size())
print(order_logistic_states.shape)
order_logistic_states.head()


review_score
reviewscore_1.0    1000
reviewscore_5.0    1000
dtype: int64
(2000, 148)


,order_id,action_0,action_1,action_2,action_3,action_4,action_5,action_6,action_7,action_8,...,action_137,action_138,action_139,action_140,action_141,action_142,action_143,action_144,action_145,review_score
0,8438989,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,CONSIGN,CONSIGN,CONSIGN,CONSIGN,...,None,None,None,None,None,None,None,None,None,reviewscore_1.0
1,134036864,CONSIGN,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,ARRIVAL,...,None,None,None,None,None,None,None,None,None,reviewscore_1.0
2,29236693,ORDERED,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,CONSIGN,...,None,None,None,None,None,None,None,None,None,reviewscore_1.0
3,87299712,ORDERED,ORDERED,ORDERED,ORDERED,ORDERED,CONSIGN,CONSIGN,DEPARTURE,DEPARTURE,...,None,None,None,None,None,None,None,None,None,reviewscore_1.0
4,19029696,ORDERED,CONSIGN,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,DEPARTURE,...,None,None,None,None,None,None,None,None,None,reviewscore_1.0


### Optimal Matching Analysis (using R)


#### 載入資料

```r
library('TraMineR')
library('cluster')
library('factoextra')
library('NbClust')
library('caret')
library('RcmdrMisc')
library('ggplot2')

# Variables
case_name = 'Cainiao'
preprocess_folderpath <- sprintf('%s/%s_preprocessed', case_name, case_name)
output_folderpath <- sprintf('%s/%s_output', case_name, case_name)
produce_date <- '20210715'
sent_day <- 10

# Assign case name
if (case_name == 'TMall') {
  file_name <- sprintf('%s_V3.2-duration_%s_%s-label_%s', produce_date, start_day, end_day, label)
  data <- read.csv(sprintf('%s/TMall_user_state_sequence_table_%s.csv', preprocess_folderpath, file_name))
  colorset <- c("gray", "forestgreen", "darkorange3", "green", "gold", "white")
} else {
  file_name <- sprintf('%s-sentday_%s', produce_date, sent_day)
  data <- read.csv(sprintf('%s/order_logistic_states-%s.csv', preprocess_folderpath, file_name))
  colorset <- c("white","aquamarine3","azure1","azure2","dodgerblue1","aquamarine1","azure3","aquamarine4","green","green3")
}

```

#### 計算狀態轉移矩陣、置換/增刪成本、相異度矩陣


```r
# Check if missing values exist
data[is.na(data)]

# Sequence Formatting
data.seq <- seqdef(data, 2:147, xtstep=10)

# Transition rates between states
data.trate <- round(seqtrate(data.seq, time.varying = FALSE), 2)
View(data.trate)
write.csv(data.trate, sprintf('%s/transition_rate_%s.csv', preprocess_folderpath, file_name))

# Substitution Cost & Indel Cost
data.seq.cost <- seqcost(data.seq, method='TRATE')
write.csv(data.seq.cost$sm, sprintf('%s/substitution_cost_matrix_%s.csv', preprocess_folderpath, file_name))

# Optimal Matching: computing distances between sequences
data.om <- seqdist(data.seq, method = 'OMspell', sm = 'TRATE', indel = "auto")  # method = {'OM', 'OMloc', 'OMslen', 'OMspell', 'OMstran'}

# output - dissimilarity matrix
write.csv(data.om, 
          sprintf('%s/dissimilarity_matrix-%s-seqdist_%s-sm_%s-indel_%s.csv', 
                  preprocess_folderpath, file_name, 'OMspell', 'TRATE', 'auto'), 
          row.names = FALSE)

```



### 分群分析


#### 搜尋最佳群數 (R)

```r
# Find optimal number of clusters, method = "silhouette", "wss"
fviz_nbclust(data.om, FUNcluster = kmeans, method = "silhouette", k.max = 8, print.summary = TRUE)
```


#### 搜尋最佳群數 (Python)

In [ ]:
if check_best_centers:
    cluster_methods = ['hcut', 'kmeans']
    OM_versions = ['OM', 'OMloc', 'OMslen', 'OMspell', 'OMstran']

    print('Start checking for best cluster center number...')
    for cluster_method in cluster_methods:
        for OM_version in OM_versions:
            silhouette_plot(file_name=file_name, OM_version=OM_version, sm_method=sm_method, indel_method=indel_method,
                            cluster_method=cluster_method, max_cluster=8)


#### 分群演算法並回標原資料集

```r
# Clustering: Hierarchical Clustering
clusterward <- agnes(data.om, diss = TRUE, method = 'ward')
cluster.result <- cutree(clusterward, k=2)

# Clustering: K-means
cluster.result <- kmeans(data.om, centers = 2)$cluster

# change cluster label name
cluster.label <- factor(cluster.result, labels = paste('Cluster'), 1:2)

# output - cluster labels add back to user state sequences data
data$kmeans_cluster <- cluster.label

write.csv(data, 
          sprintf('%s/clustered-%s-seqdist_%s-sm_%s-indel_%s-method_%s-center_%s.csv',
                  output_folderpath, file_name, 'OMspell', 'TRATE', 'auto', 'kmeans', '2'), 
          row.names = FALSE)

```

#### 序列狀態分布圖

```r
# Plots
plot_name <- sprintf('%s/cluster_distribution-%s-seqdist_%s-sm_%s-indel_%s-method_%s-center_%s', 
                     output_folderpath, file_name, 'OMspell', 'TRATE', 'auto', 'kmeans', '2')

# State distribution plot
seqdplot(data.seq, group = cluster.label.kmeans, border=NA, cpal=colorset)

```

### 計算分群指標及其統計顯著性

####  載入中間產出資料集

In [10]:
# 已標記分群標籤的訂單物流序列資料
clusteredSeqs = pd.read_csv(output_folderpath + f'clustered-{file_name}-seqdist_{OM_version}-sm_{sm_method}-indel_{indel_method}-method_{cluster_method}-center_{center}.csv')
clusteredSeqs = clusteredSeqs[['order_id', f'{cluster_method}_cluster']]
print(clusteredSeqs.shape)
clusteredSeqs.head()

# 前面預存的抽樣訂單原始資料
sampledOrderLog = pd.read_csv(preprocessed_folderpath + f'Cainiao-sampledData_reviewscore-{file_name}.csv')
sampledOrderLog = sampledOrderLog.drop(['order_date', 'logistic_order_id', 'action', 'facility_id', 'facility_type',
                                        'city_id', 'logistic_company_id', 'timestamp', 'sent_duration'], axis=1)
sampledOrderLog = sampledOrderLog[sampledOrderLog['order_id'].isin(clusteredSeqs.order_id.unique())]
sampledOrderLog['promise_speed'] = sampledOrderLog['promise_speed'].fillna(0.0)
sampledOrderLog['end_time'] = pd.to_datetime(sampledOrderLog['end_time'])
sampledOrderLog['start_time'] = pd.to_datetime(sampledOrderLog['start_time'])

assign_cols = ['pay_timestamp', 'end_time']
generate_cols = ['pay', 'receive']
sampledOrderLog = generate_time_features(df=sampledOrderLog, assign_cols=assign_cols, generate_cols=generate_cols)
print(sampledOrderLog.shape)
sampledOrderLog.head()



(2000, 2)


,order_id,kmeans_cluster
0,8438989,Cluster1
1,134036864,Cluster1
2,29236693,Cluster1
3,87299712,Cluster1
4,19029696,Cluster1


pay_month
1    25532
dtype: int64
pay_hour_range
0    1161
1    8275
2    9238
3    6858
dtype: int64
pay_day_range
0     3020
1    10805
2    11707
dtype: int64
receive_month
1     7033
2    18499
dtype: int64
receive_hour_range
0      123
1     8553
2    13832
3     3024
dtype: int64
(25532, 15)


,order_id,promise_speed,if_cainiao,Logistics_review_score,pay_hour,pay_dayOfWeek,pay_month,pay_day,pay_hour_range,pay_day_range,receive_hour,receive_dayOfWeek,receive_month,receive_day,receive_hour_range
0,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1
1,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1
2,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1
3,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1
4,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1


#### 計算分群前及分群後各群的總體指標
- 平均值
- 標準差
- 中位數


In [11]:
# 計算分群前的總體指標平均、標準差與中位數
sampledOrderLog = pd.merge(sampledOrderLog, clusteredSeqs, on='order_id', how='left')
sampledOrderLog = sampledOrderLog.rename(columns={f'{cluster_method}_cluster': 'om_cluster'})
sampledOrderLog
origin_metrics = compute_origin_metrics(sampledOrderLog)
origin_metrics.to_csv(output_folderpath + f'origin_metrics-{file_name}.csv', index=False)


# 再根據各群算出各指標的平均、標準差與中位數
cluster_metrics = compute_cluster_metrics(sampledOrderLog)
cluster_metrics
cluster_metrics.to_csv(output_folderpath + f'cluster_metrics-{file_name}-seqdist_{OM_version}-sm_{sm_method}-indel_{indel_method}-method_{cluster_method}-center_{center}.csv',
                       index=False)



,order_id,promise_speed,if_cainiao,Logistics_review_score,pay_hour,pay_dayOfWeek,pay_month,pay_day,pay_hour_range,pay_day_range,receive_hour,receive_dayOfWeek,receive_month,receive_day,receive_hour_range,om_cluster
0,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1,Cluster1
1,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1,Cluster1
2,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1,Cluster1
3,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1,Cluster1
4,19934769,0.0,1.0,1.0,12,6,1,21,1,2,12,4,2,40,1,Cluster1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25527,106694200,0.0,0.0,1.0,21,2,1,31,3,2,11,5,2,48,1,Cluster1
25528,106694200,0.0,0.0,1.0,21,2,1,31,3,2,11,5,2,48,1,Cluster1
25529,106694200,0.0,0.0,1.0,21,2,1,31,3,2,11,5,2,48,1,Cluster1
25530,106694200,0.0,0.0,1.0,21,2,1,31,3,2,11,5,2,48,1,Cluster1


,Cluster1(mean),Cluster1(std),Cluster1(median),Cluster2(mean),Cluster2(std),Cluster2(median)
是否承諾送達時限,0.102,0.518,0.0,0.009,0.152,0.0
是否從菜鳥物流倉儲出貨,0.141,0.348,0.0,0.091,0.288,0.0
物流滿意度評分,2.451,1.923,1.0,3.878,1.797,5.0
訂單成立時間(小時),14.464,5.696,15.0,14.921,5.641,15.0
訂單成立時間(星期幾),3.615,2.022,3.0,4.074,2.020,4.0
訂單成立時間(日),1.000,0.000,1.0,1.000,0.000,1.0
訂單成立時間(月),17.272,6.543,18.0,23.943,4.859,25.0
訂單成立時間(時段),1.843,0.864,2.0,1.878,0.876,2.0
訂單成立時間(日期區間),1.128,0.651,1.0,1.829,0.453,2.0
送達時間(小時),15.172,3.676,16.0,15.504,3.335,16.0


#### 計算兩群間樣本特徵分布顯著性

In [12]:
metrics_to_compare = ['Logistics_review_score', 'pay_day', 'receive_day']
for metric in metrics_to_compare:
    compute_kruskal_wallis_test(df=sampledOrderLog, metric=metric, cluster_num=center)



Kruskal-Wallis Significance Test: Logistics_review_score

Origin  v.s.  Cluster1
KruskalResult(statistic=500.1294619184082, pvalue=8.90844703296435e-111)

Cluster1  v.s.  Cluster2
KruskalResult(statistic=2752.2560605520966, pvalue=0.0)
Origin  v.s.  Cluster2
KruskalResult(statistic=1472.7637292810327, pvalue=0.0)

Kruskal-Wallis Significance Test: pay_day

Origin  v.s.  Cluster1
KruskalResult(statistic=1208.7722926361832, pvalue=7.565558311368026e-265)

Cluster1  v.s.  Cluster2
KruskalResult(statistic=6773.732266302349, pvalue=0.0)
Origin  v.s.  Cluster2
KruskalResult(statistic=3625.527322187545, pvalue=0.0)

Kruskal-Wallis Significance Test: receive_day

Origin  v.s.  Cluster1
KruskalResult(statistic=1013.5498177327054, pvalue=2.0372465317077927e-222)

Cluster1  v.s.  Cluster2
KruskalResult(statistic=5685.1308313889995, pvalue=0.0)
Origin  v.s.  Cluster2
KruskalResult(statistic=3048.0768109337837, pvalue=0.0)

